In [ ]:
#split training data up for augmentation
import os
import glob
import shutil

from sklearn.model_selection import train_test_split 

input_dir = r'C:\Users\marie\OneDrive\Desktop\Brendan\renamed files\archive (14)\train - Copy' #directory files will be taken from, making copy of training data 

file_list = glob.glob(os.path.join(input_dir, '**', '*.jpg'), recursive=True) #get files from directory and subdirectory in input directory


#directories where the data used for training and data used for augmentation will be 
train_data  = r'C:\Users\marie\OneDrive\Desktop\Brendan\renamed files\archive (14)\train_data'
aug_data = r'C:\Users\marie\OneDrive\Desktop\Brendan\renamed files\archive (14)\aug_data'

train_files, test_files = train_test_split(file_list, train_size= 5500, random_state=42) #splitting up files so approx 2000 will be used for augmentation and keep rest for training

#copy training files to training dir
for file in train_files:
    rel_path = os.path.relpath(file, input_dir) #get relative path from input directory
    dest_path = os.path.join(train_data, rel_path) #destination wherefiles are to be copied too
    os.makedirs(os.path.dirname(dest_path), exist_ok=True) #create folders in destination path
    shutil.copy2(file, dest_path) #copy files to destination path

#copy augment files to test dir
for file in test_files:
    rel_path = os.path.relpath(file, input_dir)   #get relative path from input directory
    dest_path = os.path.join(aug_data, rel_path) #destination wherefiles are to be copied too
    os.makedirs(os.path.dirname(dest_path), exist_ok=True) #create folders in destination path
    shutil.copy2(file, dest_path) #copy files to destination pat

    

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import os

rootdir = r'C:\Users\marie\OneDrive\Desktop\Brendan\renamed files\archive (14)\aug_data' #root directory containing files to augment
output_dir = r'C:\Users\marie\OneDrive\Desktop\Brendan\renamed files\archive (14)\augment_data' #output directory containing augmented files
 
#datagen = ImageDataGenerator(
  #  width_shift_range=0.1,
  #  height_shift_range=0.1,
    #zoom_range=0.1,
    #brightness_range=(0.7, 1.3),
   # rotation_range=10,
  #  horizontal_flip=True
#)

#ImageDatagenerator is applied to augment files using width and height shifts and rotation ranges
datagen = ImageDataGenerator(  
    width_shift_range=0.05,
    height_shift_range=0.05,
    rotation_range=5
)

for subdir, dirs, files in os.walk(rootdir): #go through the sub directory in the folder containing files to be augmented
    for f in files:  #look through each file wthin the sub folders
        file_path = os.path.join(subdir, f)  #file path for each folder
        img = load_img(file_path) #load image     
        x = img_to_array(img) #convert image to array for keras
        x = x.reshape((1, ) + x.shape) #reshape array to add batch dim

        i = 0 #initilaise counter

        #generate augmented images using datagen
        for batch in datagen.flow(     
            x,
            batch_size=1, #augment 1 image at a time
            save_to_dir=output_dir,      #save file to output directory    
            save_prefix=os.path.splitext(f)[0],   #perserve file name
            save_format='jpeg' #prserve format
            ):
            i += 1 
            if i >= 1:  #only make 1 aumgented file per image                    
                break